<a href="https://colab.research.google.com/github/Edsonrazorback/Colab-Gemini/blob/main/PromptCBMGO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 18.6 MB/s eta 0:00:00


In [20]:
# Importações e configurações iniciais

import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get("secret_key")
genai.configure(api_key=api_key)


In [21]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)



models/embedding-001
models/text-embedding-004


In [22]:
# Exemplo de embedding
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Titulo: O proxima geração de desenvolvedores IA e Google Workspace"
     "\n"
    "Artigo completo:\n"
    "\n"
    "Gemini API & Google AI Studio: Uma maneira acessivel de explorar e criar prototipos de aplicacoes IA generativas")

model = 'models/embedding-001'
embedding = genai.embed_content(model=model,
                                content=sample_text,
                                task_type="retrieval_document",
                                title=title)

print(embedding)



{'embedding': [0.044252567, -0.037835438, -0.023600463, 0.010069208, 0.0670061, 0.0014657151, -0.024965215, -0.020967126, 0.0650814, 0.060275037, 0.008890537, 0.021741152, -0.046959303, -0.026245475, 0.012208038, -0.01367528, 0.023423081, -0.010866226, -0.03183737, -0.00050353404, 0.015651735, 0.0049143634, -0.03132415, -0.06905188, -0.021255892, 0.020291854, 0.008740383, -0.03575055, -0.035977904, 0.03292907, -0.048898753, 0.045594342, -0.037807856, 0.015649352, -0.030075572, -0.034687072, -0.026586592, -0.051727936, 0.0018744981, -0.002408094, 0.004736527, -0.08545924, -0.009283671, 0.019859537, -0.006615628, -0.029296502, 0.03962424, 0.047789652, 0.022256894, -0.054476816, 0.022287818, 0.020343492, 0.070406064, -0.041656338, -0.0050832583, -0.0067521185, 0.028848052, -0.037730813, 0.022484876, 0.004231023, -0.003236667, 0.023091022, -0.017504076, 0.05668885, 0.015626933, -0.0539055, -0.03659329, -0.002532583, 0.017115453, 0.05347278, 0.009403887, 0.007594928, 0.063970976, -0.0712878

In [23]:
DOCUMENT1 = {
    "titulo": "Operating the Climate Control System",
    "conteudo": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}
DOCUMENT2 = {
    "titulo": "Touchscreen",
    "conteudo": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}
DOCUMENT3 = {
    "titulo": "Shifting Gears",
    "conteudo": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [24]:
df = pd.DataFrame(documents)
df.columns = ['titulo', 'conteudo']
df

,titulo,conteudo
0,Operating the Climate Control System,Your Googlecar has a climate control system th...
1,Touchscreen,Your Googlecar has a large touchscreen display...
2,Shifting Gears,Your Googlecar has an automatic transmission. ...


In [25]:
model = "models/embedding-001"


In [26]:
def embed_fn (title, text):
   return genai.embed_content(model=model,
                                content=text,
                                task_type="retrieval_document",
                                title=title)["embedding"]

In [27]:
df["Embeddings"] = df.apply(lambda row: embed_fn (row["titulo"], row["conteudo"]), axis=1)
df

,titulo,conteudo,Embeddings
0,Operating the Climate Control System,Your Googlecar has a climate control system th...,"[-0.033361107, -0.021217084, -0.049581926, -0...."
1,Touchscreen,Your Googlecar has a large touchscreen display...,"[0.009660736, -0.030662702, -0.017281422, -0.0..."
2,Shifting Gears,Your Googlecar has an automatic transmission. ...,"[-0.04270796, -0.007160868, -0.03242516, -0.02..."


In [38]:
def gerar_e_buscar_consulta(consulta, base, model):
   embedding_da_consulta = genai.embed_content(model=model,
                                content=consulta,
                                task_type="RETRIEVAL_QUERY")["embedding"]

   produto_escalares = np.dot(np.stack(df["Embeddings"]),embedding_da_consulta)

   indice = np.argmax(produto_escalares)
   return df.iloc[indice]["conteudo"]

In [36]:
consulta = "googlecar?"

trecho = gerar_e_buscar_consulta(consulta,df,model)
print(trecho)


Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.


In [30]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5,
}

In [37]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)


Seu Googlecar tem uma telona sensível ao toque irada que te dá acesso a um monte de coisa maneiro, tipo GPS, música e até controle do ar-condicionado. Pra usar, é só dar um toque no ícone que você quer. Por exemplo, toca no "GPS" pra saber como chegar no rolê ou no "Música" pra botar o som que você curte.
